In [349]:
import numpy as np
import matplotlib.pyplot as plt


**Example: Forward propagation**

$$Z^{[1]} = W^{[1]} X + b^{[1]}$$
$$A^{[1]} = g_{\text{ReLU}}(Z^{[1]}))$$
$$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$$
$$A^{[2]} = g_{\text{ReLU}}(Z^{[2]}))$$
$$Z^{[3]} = W^{[3]} A^{[2]} + b^{[3]}$$
$$A^{[3]} = g_{\text{softmax}}(Z^{[3]})$$

In [350]:
class Linear:
    def __init__(self,n_inputs,n_outputs):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.weights = np.random.rand(n_inputs,n_outputs)
        self.biases = np.zeros((1,n_outputs))
    def forward(self,X):
        self.X = X
        self.Z = self.X.dot(self.weights) + self.biases
        return self.Z
    def backward(self,Layer):
        dz_dw = self.X
        dz_db = np.ones_like(Layer.dbackward)
        dz_dA = self.weights
        self.dweights = dz_dw.T.dot(Layer.dbackward)
        self.dbiases = np.sum(Layer.dbackward,axis=0,keepdims=True)
        self.dbackward =Layer.dbackward.dot(dz_dA.T)
class ReLU:
    def forward(self,X):
        self.A = np.maximum(0,X)
        return self.A
        
    def backward(self,Layer):
        dA_dz = np.where(Layer.dbackward>0,1,0)
        self.dbackward = np.multiply(Layer.dbackward,dA_dz)
    
class SoftMax:
    def __init__(self,n_inputs,n_outputs):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
    def forward(self,X):
        X = np.exp(X) / np.sum(np.exp(X),axis=1,keepdims=True)
        return X

class MSELoss:
    def forward(self,Y_pred,Y_true):
        self.Y_pred = Y_pred
        self.Y_true = Y_true
        self.loss = ((Y_pred-Y_true)**2).mean()
        return self.loss
    def backward(self):
        self.dbackward = 2* (self.Y_pred - self.Y_true)
        


class OptimizerSGD:
    def __init__(self,lr=0.1):
        self.lr = lr
    def update_params(self,Layer):
        Layer.weights -= self.lr * Layer.dweights
        Layer.biases -= self.lr * Layer.dbiases


In [351]:
class NeuralNetwork:
    def __init__(self,n_inputs,n_outputs):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.linear1 = Linear(n_inputs=self.n_inputs,n_outputs=64)
        self.relu1 = ReLU()
        self.linear2 = Linear(n_inputs=64,n_outputs=128)
        self.relu2 = ReLU()
        self.linear3 = Linear(n_inputs=128,n_outputs=self.n_outputs)
        self.loss = MSELoss()
    def forward(self,X):
        X = self.linear1.forward(X)
        X = self.relu1.forward(X)
        X = self.linear2.forward(X)
        X = self.relu2.forward(X)
        X = self.linear3.forward(X)
        return X
    
    def calculate_loss(self,y_pred,y_true):
        self.loss.forward(y_pred,y_true)
    def backward(self):
        self.loss.backward()
        self.linear3.backward(self.loss)
        self.relu2.backward(self.linear3)
        self.linear2.backward(self.relu2)
        self.relu1.backward(self.linear2)
        self.linear1.backward(self.relu1)
    
    def update(self):
        optim = OptimizerSGD()
        optim.update_params(self.linear1)
        optim.update_params(self.linear2)
        optim.update_params(self.linear3)
    
    def fit(self,X,y,epochs = 1000):
        for i in range(epochs):
            y_pred = self.forward(X)
            self.calculate_loss(y_pred,y)
            self.backward()
            self.update()
            print(y_pred)



In [352]:
X = np.array([[1,2,3,4,5,6]])
y = np.array([[24,67,234,634]])
model = NeuralNetwork(X.shape[1],y.shape[1])

In [353]:
model.fit(X,y,epochs=150)

[[20616.85240773 20805.87032611 19019.1595783  22869.48062792]]
[[-4118.57048155 -4147.77406522 -3757.03191566 -4447.09612558]]
[[-3290.05638524 -3304.81925218 -2958.82553253 -3430.87690047]]
[[-2627.24510819 -2630.45540174 -2320.26042602 -2617.90152037]]
[[-2096.99608655 -2090.96432139 -1809.40834082 -1967.5212163 ]]
[[-1672.79686924 -1659.37145711 -1400.72667265 -1447.21697304]]
[[-1333.43749539 -1314.09716569 -1073.78133812 -1030.97357843]]
[[-1061.94999631 -1037.87773255  -812.2250705   -697.97886275]]
[[-844.75999705 -816.90218604 -602.9800564  -431.5830902 ]]
[[-671.00799764 -640.12174883 -435.58404512 -218.46647216]]
[[-532.00639811 -498.69739907 -301.6672361   -47.97317773]]
[[-420.80511849 -385.55791925 -194.53378888   88.42145782]]
[[-331.84409479 -295.0463354  -108.8270311   197.53716626]]
[[-260.67527583 -222.63706832  -40.26162488  284.829733  ]]
[[-203.74022067 -164.70965466   14.5907001   354.6637864 ]]
[[-158.19217653 -118.36772373   58.47256008  410.53102912]]
[[-121.7